<a href="https://colab.research.google.com/github/usmanmalik07/Adept-Internship/blob/main/2Layer_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

# Load the Iris dataset
iris = load_iris()
data = iris.data
targets = iris.target

# Standardize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Split the data into training and test sets
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
train_data = torch.tensor(train_data, dtype=torch.float32)
train_targets = torch.tensor(train_targets, dtype=torch.long)
test_data = torch.tensor(test_data, dtype=torch.float32)
test_targets = torch.tensor(test_targets, dtype=torch.long)

# Create DataLoader for training and test sets
batch_size = 16
train_dataset = TensorDataset(train_data, train_targets)
test_dataset = TensorDataset(test_data, test_targets)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [18]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Second fully connected layer

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply ReLU activation to the output of the first layer
        x = self.fc2(x)  # The output layer
        return x


In [19]:
# Hyperparameters
input_size = 4  # Number of features in the Iris dataset
hidden_size = 10
output_size = 3  # Number of classes in the Iris dataset
learning_rate = 0.001
num_epochs = 100

# Instantiate the model, loss function, and optimizer
model = TwoLayerNet(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [20]:
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.8727
Epoch [20/100], Loss: 0.6748
Epoch [30/100], Loss: 0.6187
Epoch [40/100], Loss: 0.1779
Epoch [50/100], Loss: 0.1613
Epoch [60/100], Loss: 0.2073
Epoch [70/100], Loss: 0.1617
Epoch [80/100], Loss: 0.2784
Epoch [90/100], Loss: 0.1639
Epoch [100/100], Loss: 0.2015


In [21]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for data, targets in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')


Accuracy of the model on the test set: 100.00%


In [22]:
# Making predictions on new data
new_data = torch.tensor([[5.1, 3.5, 1.4, 0.2], [6.7, 3.1, 4.7, 1.5]], dtype=torch.float32)  # Example new data points
new_data = scaler.transform(new_data)  # Apply the same scaling
new_data = torch.tensor(new_data, dtype=torch.float32)

model.eval()
with torch.no_grad():
    outputs = model(new_data)
    _, predicted = torch.max(outputs, 1)
    print(f'Predicted classes for the new data points: {predicted.numpy()}')

Predicted classes for the new data points: [0 1]
